# Tool Use with Amazon Nova

In this notebook, we will explore a few ways to use the Amazon Nova Models for Tool Use

In [ ]:
# Run this cell to install the required packages if you haven't already done so.
%pip install -r requirements.txt

In [ ]:
# Execute this cell to restart kernel
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [ ]:
# Create OpenAI Client with Nova API
from openai import OpenAI
import os
from dotenv import load_dotenv
import json

#Set up your environment variables
api_key = os.getenv("NOVA_API_KEY")
base_url = "https://api.nova.amazon.com/v1"

# Create OpenAI client
client = OpenAI(api_key=api_key, base_url=base_url)

# Configure model_id
model_id = 'nova-2-lite-v1' # You can change this to any other Nova model available to you

## Using Built-in Tools with Amazon Nova 

In [ ]:
## define the message and we add system role to influence the response

messages = [
    {
      "role": "system",
      "content": "You are an expert python programmer and you add comments to improve readabilty"
    },
    {
      "role": "user",
      "content": "Write a properly formatted python function to evaluate 7**6 and print the result"
    }
]

## Make the API call 

response = client.chat.completions.create(
    model=model_id,
    messages=messages,
    tool_choice="auto",
    temperature=0.9,
    extra_body={ 
        "system_tools" : ["nova_code_interpreter"]
        }
)

print(response.choices[0].message.content)

## Defining a custom tool for Nova

In [ ]:
## define the message and we add system role to influence the response

def evaluate_expression(equation):
    return eval(equation)

messages = [
    {
      "role": "user",
      "content": "Calculate 7^6 using the calculator tool"
    }
]

# Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "evaluate_expression",
            "description": "A calculator tool that can execute a math equation",
            "parameters": {
                "type": "object",
                "properties": {
                    "equation": {
                        "type": "string",
                        "description": "The equation to be evaluated"
                    }
                },
                "required": ["equation"]
            }
        }
    },
]

## Make the API call 

response = client.chat.completions.create(
    model=model_id,
    messages=messages,
    tool_choice="required", #none, auto, required
    tools=tools,
    temperature=0.9,
    stream=False
)

tool_function_name = response.choices[0].message.tool_calls[0].function.name
tool_function_args = response.choices[0].message.tool_calls[0].function.arguments
print(f"Tool returned Function name: {tool_function_name}")
print(f"Tool returned Function parameters: {tool_function_args}")

In the above example, the model should have selected the correct tool function and passed it the correct args. The next step, we will "execute" that tool call and return the result.

In [ ]:
if tool_function_name == "evaluate_expression":
    print(f"Tool returned Function result: {evaluate_expression(json.loads(tool_function_args)['equation'])}")

## Chaining Tools
The previous example illustrated a primitive tool call example. In a real use case, you might want to chain these calls, taking a tool response from one tool and use it in another tool call, which is orchestrated by the model. In this section, we will chain the tool sequence, where output of first auto tool is used by the next user defined tool in the chain

In [ ]:
## Use Nova Auto tool functionality to answer the user query search tool to find answer
messages = [
    {
      "role": "user",
      "content": "What is the distance between earth and sun in miles. Only respond in just numbers. Exclude explanation or references."
    }
]

## Make the API call 

response = client.chat.completions.create(
    model=model_id,
    messages=messages,
    tool_choice="auto",
    temperature=0.8, 
    stream=False
)


print(response.choices[0].message.content)

In [ ]:
## append the response to messages list 
messages.append({
    "role": "assistant",
    "content": response.choices[0].message.content
})

## append the followup user query 
messages.append({
      "role": "user",
      "content": """Use the calculator tool to convert the response from miles to kilometers. 
                    Responsd in this format: 
                    Distance {distance in miles} miles is {distance in kilometers} kilometers\n"""
    })
print(json.dumps(messages, indent=4))

In [ ]:
## define a custom tool to conver miles to kilometers
def miles_to_km(miles):
    return int(miles) * 1.60934


# Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "miles_to_km",
            "description": "A calculator tool to convert from miles to Kilometers",
            "parameters": {
                "type": "object",
                "properties": {
                    "miles": {
                        "type": "string",
                        "description": "The miles"
                    }
                },
                "required": ["miles"]
            }
        }
    },
]

## make the API call 
response = client.chat.completions.create(
    model=model_id,
    messages=messages,
    tool_choice="required",
    tools=tools,
    temperature=1.0, ## this being calculation, we want same reponse everytime
    stream=False
)



In [ ]:
## Extract tool name and arguments returned by Nova and use it to call custom function
tool_function_name = response.choices[0].message.tool_calls[0].function.name
tool_function_args = response.choices[0].message.tool_calls[0].function.arguments
if tool_function_name == "miles_to_km":
    print(f"Tool returned Function result : \n{json.loads(tool_function_args)['miles']} miles is {miles_to_km(json.loads(tool_function_args)['miles'])} kilometers")